In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric import seed_everything
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
import torch_geometric
from tqdm import tqdm, trange
from torcheval.metrics import MulticlassAccuracy
from torcheval.metrics import BinaryAUROC
import numpy as np

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# DEVICE = torch.device('cuda')
print(DEVICE)

/home/tsou/.conda/envs/pyg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# get feature from csv
# RAW_data = pd.read_csv('data/adult.csv')
# CAT = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']
# NUM = ['age','fnlwgt','educational-num','capital-gain','capital-loss','hours-per-week']
# LABEL = 'income'
RAW_data = pd.read_csv('data/compass_old.csv')
CAT=['sex','age_cat','race','c_charge_degree','decile_score.1','score_text','v_type_of_assessment','v_decile_score','v_score_text']
NUM=['age','juv_fel_count','juv_misd_count','juv_other_count','priors_count','days_b_screening_arrest','c_days_from_compas','end']
LABEL = 'is_recid'
# convert categorical data to ordinal data
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
data_pd = RAW_data.copy()
data_pd[CAT] = enc.fit_transform(RAW_data[CAT])
# data_pd = pd.get_dummies(RAW_data, columns=CAT, dtype=float)
# label to category
data_pd[LABEL] = data_pd[LABEL].astype('category').cat.codes

# realign data to num + cat
data_pd = data_pd[NUM + CAT + [LABEL]]

# caculate unique value of each categorical feature
cat_num = [len(data_pd[col].unique()) for col in CAT]

# normalize numerical data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_pd[NUM] = scaler.fit_transform(data_pd[NUM])

# convert data to tensor
x = torch.tensor(data_pd.drop(columns=[LABEL]).values, dtype=torch.float, device=DEVICE)  # [48842, 108]
y = torch.tensor(data_pd[LABEL].values, dtype=torch.long, device=DEVICE) # [48842]
print(x.shape, y.shape)
print(cat_num)
data_pd

torch.Size([16644, 17]) torch.Size([16644])
[2, 3, 6, 13, 10, 3, 1, 10, 3]


,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,end,sex,age_cat,race,c_charge_degree,decile_score.1,score_text,v_type_of_assessment,v_decile_score,v_score_text,is_recid
0,-0.763142,-0.17116,-0.195994,-0.249708,-0.769173,-0.067721,-0.155864,0.360236,1.0,0.0,0.0,3.0,3.0,1.0,0.0,3.0,1.0,0
1,0.277735,-0.17116,-0.195994,-0.249708,-0.399623,-0.067721,-0.155864,-1.493342,0.0,0.0,2.0,2.0,4.0,2.0,0.0,1.0,1.0,0
2,-0.069224,-0.17116,-0.195994,-0.249708,-0.030074,-0.744908,0.051069,1.034535,1.0,0.0,2.0,6.0,5.0,2.0,0.0,4.0,2.0,0
3,0.884913,-0.17116,-0.195994,-0.249708,-0.030074,-0.190846,-0.118240,0.366177,0.0,0.0,2.0,2.0,3.0,1.0,0.0,1.0,1.0,0
4,-0.849881,-0.17116,-0.195994,7.344876,-0.584398,-0.350908,-0.069329,1.287026,0.0,2.0,2.0,7.0,3.0,1.0,0.0,3.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16639,-1.110101,-0.17116,-0.195994,-0.249708,-0.769173,-0.067721,-0.155864,1.319701,1.0,2.0,2.0,7.0,5.0,2.0,0.0,6.0,2.0,1
16640,-0.329443,-0.17116,-0.195994,-0.249708,-0.769173,-0.067721,-0.155864,-0.245741,1.0,0.0,0.0,7.0,1.0,1.0,0.0,1.0,1.0,1
16641,-0.329443,-0.17116,-0.195994,-0.249708,-0.769173,-0.067721,-0.155864,0.235476,1.0,0.0,0.0,7.0,1.0,1.0,0.0,1.0,1.0,1
16642,-0.936621,-0.17116,-0.195994,-0.249708,-0.399623,-0.080033,-0.152102,-1.053711,0.0,2.0,3.0,3.0,3.0,1.0,0.0,3.0,1.0,1


In [3]:
class feature_improtance_extractor():
    def __init__(self):
        self.feature_importance = []
        self.iter = 0
        pass
    def update(self, new_feature_importance):
        if self.iter == 0:
            self.feature_importance = new_feature_importance
        else:
            self.feature_importance += new_feature_importance
        self.iter += 1
        return
    
    def get(self):
        return (self.feature_importance / self.iter)
    
    def reset(self):
        self.feature_importance = []
        self.iter = 0
        return
extractor = feature_improtance_extractor()

In [21]:
class K_graph(torch.nn.Module):
    def __init__(self, NUM, CAT, LABEL, cat_num):
        super(K_graph, self).__init__()
        '''
        num_cols: number of numerical columns
        cat_cols: number of categorical columns
        label_cols: number of label columns
        cat_num: number of unique value of each categorical columns
        '''
        self.hidden_dim = 128
        # order: num -> cat -> label
        self.num_cols = len(NUM)
        self.cat_cols = len(CAT)
        self.label_cols = len(LABEL)
        self.number_of_columns = self.num_cols + self.cat_cols 
        self.K = round(self.number_of_columns*0.5)
        
        # numerical feature
        self.num_embeddings = torch.nn.ModuleList([torch.nn.Linear(1, self.hidden_dim) for i in range(self.num_cols)])
        # categorical feature
        self.cat_embeddings = torch.nn.ModuleList([torch.nn.Embedding(cat_num[i], self.hidden_dim) for i in range(self.cat_cols)])
        
        self.prediction = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim *( self.K + self.number_of_columns), self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(self.hidden_dim),
            torch.nn.Linear(self.hidden_dim, self.label_cols + 1)
        )
        
        # feature importance learning
        self.feature_importance_learners = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(self.hidden_dim),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(self.hidden_dim, 1),
        ) 
        
        # graph convolution layers
        self.conv_GCN_input = torch_geometric.nn.GCNConv(self.number_of_columns*self.hidden_dim, self.hidden_dim)
        # self.conv_GCN_input = torch_geometric.nn.GCNConv(self.hidden_dim, self.hidden_dim)
        # self.conv_1_input = torch_geometric.nn.GATConv(self.number_of_columns*self.hidden_dim, self.hidden_dim)
        self.conv_GCN_2 = torch_geometric.nn.GCNConv(self.hidden_dim, self.hidden_dim)
        
        # self.transform = torch.nn.Linear(self.number_of_columns*self.hidden_dim, self.hidden_dim)
        
    def forward(self, input_data, epoch = -1):
        
        # make feature embedding
        num_data = input_data[:,:self.num_cols].unsqueeze(-1).unsqueeze(-1) 
        feature_embedding_num = torch.cat([self.num_embeddings[i](num_data[:,i]) for i in range(self.num_cols)], dim=1).reshape(len(input_data), -1) # [batch_size, num_cols * hidden_dim]
        feature_embedding_num = torch.nn.ReLU()(feature_embedding_num)
        feature_embedding_num = torch.layer_norm(feature_embedding_num, feature_embedding_num.shape)
        # categorical feature
        feature_embedding_cat = torch.cat([self.cat_embeddings[i](input_data[:,self.num_cols+i].long()) for i in range(self.cat_cols)], dim=1) # [batch_size, cat_cols * hidden_dim]
        feature_embedding_cat = torch.layer_norm(feature_embedding_cat, feature_embedding_cat.shape)
        # concat
        feature_embedding = torch.cat((feature_embedding_num, feature_embedding_cat), dim=1) # [batch_size, (num_cols + cat_cols) * hidden_dim]
        # feature_embedding = feature_embedding.reshape((len(input_data), self.number_of_columns, -1)) # [batch_size, (num_cols + cat_cols), hidden_dim]
        
        # feature importance learning
        feature_importance = torch.cat([self.feature_importance_learners(feature_embedding[:,i*self.hidden_dim:(i+1)*self.hidden_dim]) for i in range(self.number_of_columns)], dim=1) # [batch_size, num_cols + cat_cols, 1]
        # print(feature_importance)
        feature_importance = torch.layer_norm(feature_importance, feature_importance.shape)
        # feature_importance = torch.softmax(feature_importance, dim=1) # [batch_size, num_cols + cat_cols, 1]
        # print(feature_importance.shape)
        # print(feature_importance.sum(dim=1))
        # print(feature_importance)
        
        # weighted feature embedding 
        feature_embedding = feature_embedding.reshape((len(input_data),self.number_of_columns, -1)) * feature_importance.unsqueeze(-1) # [batch_size, (num_cols + cat_cols) * hidden_dim]
        feature_embedding = feature_embedding.reshape((len(input_data), -1)) # [batch_size, (num_cols + cat_cols) * hidden_dim]
        
        # top K feature importance
        K = self.K
        value, indices = torch.topk(feature_importance, K) # (value: [batch_size, k], indices: [batch_size, k])
        mask = torch.zeros_like(feature_importance, device=DEVICE)
        mask.scatter_(1, indices, 1)
        # importance_topK = torch.where(mask > 0, feature_importance, torch.zeros(feature_importance.shape,device=DEVICE)) # [batch_size, cols]
        importance_topK = torch.where(mask > 0, feature_importance, torch.empty(feature_importance.shape,device=DEVICE).fill_(-1e9)) # [batch_size, cols]
        importance_topK = torch.softmax(importance_topK, dim=1) # [batch_size, cols]
        # importance_topK = torch.stack([importance_topK.clone() for _ in range(self.number_of_columns)], dim=0) # [cols, batch_size, cols]
        
        extractor.update(feature_importance.sum(dim=0)/len(input_data))
        del feature_embedding_num, feature_embedding_cat, num_data
        del mask, feature_importance, value, indices
        
        
        processed_data = []
        processed_indices = []
        for target_col in range(self.number_of_columns):
            importance_topK_current = importance_topK.clone()# [batch_size, cols] 
            indices = importance_topK_current.T[target_col].nonzero().T[0] # selected samples' indices  
            
            if indices.shape[0] == 0:
                continue
            
            importance_topK_current = importance_topK_current[importance_topK_current.T[target_col]>0]# [????, cols]
            
            # for target column, set its importance to 0. so that it will not be fully connected graph
            # copy target column
            tmp = torch.clone(importance_topK_current[:,target_col]) # [????], save for future weighted sum
            importance_topK_current[:,target_col] = 0 # [????, cols]
            # multiply to get weighted adj
            weighted_adj = torch.matmul(importance_topK_current, importance_topK_current.T) # [batch_size, cols] * [cols, batch_size] = [batch_size, batch_size]
            # prune the diagonal
            weighted_adj = weighted_adj - torch.diag(weighted_adj.diagonal())

            # construct graph
            edge_index = weighted_adj.nonzero().T  # [2, num_edges]
            edge_wight = weighted_adj[edge_index[0], edge_index[1]] # [num_edges]
            edge_wight = torch.softmax(edge_wight, dim=0)

            
            if False:
                print('in graph', target_col, 'nodes:', len(indices), 'edges:', len(edge_wight),'ratio', len(edge_wight)/(len(indices)**2+0.000001))
            
            # print(edge_wight)
            # importance_topK_current[:,target_col] = tmp # [????, cols]
            
            features = (feature_embedding[indices]) # [????, cols*hidden_dim]
            # features = (feature_embedding.reshape(len(input_data),self.number_of_columns,-1)[indices][:,target_col,:]) # [????, hidden_dim]
            # print(features.shape)

            # construct graph 
            data = Data(x=features, edge_index=edge_index, edge_weight=edge_wight, indices=indices) 
            
            del features, edge_index, edge_wight, weighted_adj, importance_topK_current, tmp
            
            # apply GCN
            x = self.conv_GCN_input(data.x, data.edge_index, data.edge_weight)  # [???, hidden_dim]
            # x = self.conv_1_input(data.x, data.edge_index)  # [???, hidden_dim]
            x = torch.relu(x)
            x = torch.layer_norm(x, x.shape) # [???, hidden_dim]
            x = torch.nn.Dropout(p=0.5)(x)
            x = self.conv_GCN_2(x, data.edge_index, data.edge_weight)  # [???, hidden_dim]
            x = torch.relu(x)
            x = torch.layer_norm(x, x.shape)

            processed_data.append(x)
            processed_indices.append(indices)
        
        processed_data = torch.cat(processed_data, dim=0) 
        processed_indices = torch.cat(processed_indices, dim=0)
        # print(processed_indices)
        # print(processed_indices.argsort())
        # print(processed_indices[processed_indices.argsort()])
        processed_data = processed_data[processed_indices.argsort()] # ???????
        processed_data = torch.split(processed_data, self.K) # ?????????
        processed_data = torch.stack(list(processed_data), dim=0) # ???????????
        # processed_data = torch.sum(list(processed_data), dim=0) # ???????????

        # cat residual
        processed_data = torch.cat((processed_data, feature_embedding.reshape((len(input_data),self.number_of_columns,-1))), dim=1) # [batch_size, K+cols , hidden_dim]
        
        # make prediction
        prediction = self.prediction(processed_data.reshape(processed_data.shape[0],-1))
        # prediction = self.prediction(feature_embedding)
        
        
        return prediction


In [22]:
the_model = K_graph(NUM, CAT, [LABEL], cat_num).to(DEVICE)
optimizer = torch.optim.SGD(the_model.parameters(), lr=0.001)

# optimizer.step()
data_count = 5
# random pick data
indices = torch.randperm(len(x))[:data_count]
train_data = x[indices]
train_label = y[indices]

for i in range(5):
    
    optimizer.zero_grad()
    output = the_model(train_data[:data_count], epoch=200)
    loss = torch.nn.functional.cross_entropy(output, train_label[:data_count])
    loss.backward()
    # print(((the_model.feature_importance_learners.grad).abs().max(dim=1)[0]))
    optimizer.step()
    
    print('-----------------------------------------')


tensor([[0.0000, 0.0636],
        [0.0636, 0.0000]], device='cuda:0', grad_fn=<SubBackward0>)
tensor([[0.0000, 0.0364, 0.0268],
        [0.0364, 0.0000, 0.0643],
        [0.0268, 0.0643, 0.0000]], device='cuda:0', grad_fn=<SubBackward0>)
tensor([[0.0000, 0.0533, 0.0912, 0.0317, 0.0592],
        [0.0533, 0.0000, 0.0367, 0.0309, 0.0895],
        [0.0912, 0.0367, 0.0000, 0.0864, 0.0430],
        [0.0317, 0.0309, 0.0864, 0.0000, 0.0289],
        [0.0592, 0.0895, 0.0430, 0.0289, 0.0000]], device='cuda:0',
       grad_fn=<SubBackward0>)
tensor([[0.0000, 0.0363, 0.0942],
        [0.0363, 0.0000, 0.0860],
        [0.0942, 0.0860, 0.0000]], device='cuda:0', grad_fn=<SubBackward0>)
tensor([[0.]], device='cuda:0', grad_fn=<SubBackward0>)
tensor([[0.]], device='cuda:0', grad_fn=<SubBackward0>)
tensor([[0.0000, 0.0268, 0.0362],
        [0.0268, 0.0000, 0.0934],
        [0.0362, 0.0934, 0.0000]], device='cuda:0', grad_fn=<SubBackward0>)
tensor([[0.0000, 0.0987],
        [0.0987, 0.0000]], device='cu

In [6]:
import torchviz
plot = torchviz.make_dot(loss, params=dict(the_model.named_parameters()))
plot.render("MLP+FI", format="png")

'MLP+FI.png'

In [7]:

def train_epoch(model, optimizer, datas, batch_size, epoch):
    train_data, train_label, validation_data, validation_label = datas
    
    # slice data into batch
    train_data = torch.split(train_data, batch_size)
    train_label = torch.split(train_label, batch_size)
    validation_data = torch.split(validation_data, batch_size)
    validation_label = torch.split(validation_label, batch_size)

    # losses and metrics
    batch_loss = 0
    train_acc = MulticlassAccuracy(num_classes=2).to(DEVICE)
    train_auc = BinaryAUROC().to(DEVICE)
    valid_acc = MulticlassAccuracy(num_classes=2).to(DEVICE)
    valid_auc = BinaryAUROC().to(DEVICE)
    
    # train the model
    stepper = trange(len(train_data))
    for i in stepper:
        stepper.set_description(f'Epoch {epoch}')
        
        optimizer.zero_grad()
        output = model(train_data[i], epoch=epoch)
        loss = torch.nn.functional.cross_entropy(output, train_label[i]) * model.number_of_columns
        loss.backward()
        optimizer.step()
        batch_loss += loss.item()
        
        # metrics
        preds = output.softmax(dim=1)
        true = torch.nn.functional.one_hot(train_label[i], num_classes=2).to(DEVICE)
        train_acc.update(torch.argmax(preds, 1),true.T[1])
        train_auc.update(preds.T[0],true.T[0])
        
        # at the end of epoch, print result and validate the model
        if i == len(train_data) - 1:
            train_acc = train_acc.compute()
            train_auc = train_auc.compute()
            stepper.set_postfix({'loss': round(batch_loss/(i+1), 3), 'acc': round(train_acc.item(), 3), 'AUC': round(train_auc.item(), 3)})
            stepper.update()
        
            with torch.no_grad():
                for i in range(len(validation_data)):
                    output = model(validation_data[i], epoch=200)
                    # loss = torch.nn.functional.cross_entropy(output, validation_label[i])
                    preds = output.softmax(dim=1)
                    true = torch.nn.functional.one_hot(validation_label[i], num_classes=2).to(DEVICE)
                    valid_acc.update(torch.argmax(preds,1),true.T[1])
                    valid_auc.update(preds.T[0],true.T[0])
                stepper.set_postfix({'loss': round(batch_loss/(i+1), 3), 'acc': round(train_acc.item(), 3), 'AUC': round(train_auc.item(), 3), 'val_acc': round(valid_acc.compute().item(), 3), 'val_AUC': round(valid_auc.compute().item(), 3)})



In [8]:
def overall_train(x, y, seed=0):
    # hyperparameter
    epoch = 50
    batch_size = 1000
    seed_everything(seed)
    
    # shuffle data
    indices = torch.randperm(len(x))
    x = x[indices]
    y = y[indices]
    # slice data into train and test and validation
    train_ratio = 0.7
    validation_ratio = 0.1
    train_data = x[:int(len(x)*train_ratio)]
    train_label = y[:int(len(x)*train_ratio)]
    validation_data = x[int(len(x)*train_ratio):int(len(x)*(train_ratio+validation_ratio))]
    validation_label = y[int(len(x)*train_ratio):int(len(x)*(train_ratio+validation_ratio))]
    test_data = x[int(len(x)*(train_ratio+validation_ratio)):]
    test_label = y[int(len(x)*(train_ratio+validation_ratio)):]

    # build model and optimizer
    the_model = K_graph(NUM, CAT, [LABEL], cat_num).to(DEVICE)
    optimizer = torch.optim.SGD(the_model.parameters(), lr=0.001)
    
    # train the model
    datas = (train_data, train_label, validation_data, validation_label)
    for i in range(epoch):
        train_epoch(the_model, optimizer, datas, batch_size, epoch=i+1)
        print(extractor.get())
        extractor.reset()
    
    # test the model
    with torch.no_grad():
        test_data = torch.split(test_data, batch_size)
        test_label = torch.split(test_label, batch_size)
        for i in range(len(test_data)):
            output = the_model(test_data[i], epoch=200)
            preds = output.softmax(dim=1)
            true = torch.nn.functional.one_hot(test_label[i], num_classes=2).to(DEVICE)
            test_acc = MulticlassAccuracy(num_classes=2).to(DEVICE)
            test_auc = BinaryAUROC().to(DEVICE)
            test_acc.update(torch.argmax(preds,1),true.T[1])
            test_auc.update(preds.T[0],true.T[0])

        print('test_acc:', test_acc.compute().item())
        print('test_auc:', test_auc.compute().item())
        print('-----------------------------------------')

In [14]:
seed_set = [9,90,900,9000,90000]
overall_train(x, y, seed=9)

Epoch 1:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 12/12 [00:03<00:00,  3.63it/s, loss=74.9, acc=0.593, AUC=0.62, val_acc=0.658, val_AUC=0.712]


tensor([ 0.0910, -0.2222, -0.3857,  0.0628,  0.3273, -0.0124,  0.0954, -0.8044,
         0.1037,  0.4230,  0.2478,  0.3863, -0.0551,  0.1489,  0.0644,  0.1103,
        -0.5810], device='cuda:0', grad_fn=<DivBackward0>)


Epoch 2: 100%|██████████| 12/12 [00:03<00:00,  3.63it/s, loss=64, acc=0.657, AUC=0.709, val_acc=0.669, val_AUC=0.728]


tensor([ 0.1120, -0.4597, -0.3330,  0.1018,  0.8076, -0.2562,  0.2192, -1.0856,
        -0.1338,  0.2776,  0.3291,  0.5850,  0.0904,  0.2498,  0.1983,  0.2154,
        -0.9181], device='cuda:0', grad_fn=<DivBackward0>)


...

tensor([ 0.1954, -0.8043, -0.1846,  0.0387,  1.7375, -0.4665, -0.2932, -1.0431,
        -0.5464,  0.4019,  0.6375,  0.3966,  0.0518,  0.3247,  0.2073,  0.2274,
        -0.8807], device='cuda:0', grad_fn=<DivBackward0>)


Epoch 49: 100%|██████████| 12/12 [00:03<00:00,  3.61it/s, loss=57.7, acc=0.709, AUC=0.779, val_acc=0.702, val_AUC=0.769]


tensor([ 0.2019, -0.8125, -0.1797,  0.0366,  1.7382, -0.4702, -0.3154, -1.0306,
        -0.5427,  0.4064,  0.6408,  0.3956,  0.0442,  0.3377,  0.2059,  0.2267,
        -0.8830], device='cuda:0', grad_fn=<DivBackward0>)


Epoch 50: 100%|██████████| 12/12 [00:03<00:00,  3.62it/s, loss=57.1, acc=0.713, AUC=0.784, val_acc=0.706, val_AUC=0.777]


tensor([ 0.1809, -0.7920, -0.1780,  0.0283,  1.7456, -0.4753, -0.3254, -1.0185,
        -0.5421,  0.4120,  0.6570,  0.3937,  0.0384,  0.3390,  0.1879,  0.2247,
        -0.8761], device='cuda:0', grad_fn=<DivBackward0>)
test_acc: 0.7051671743392944
test_auc: 0.7808573540280858
-----------------------------------------


In [15]:
sum([0.7051671743392944,
     0.7082067131996155,
     0.7051671743392944,
     0.7264437675476074,
     0.7325227856636047
     ])/5

0.7155015230178833

In [16]:
sum([0.7808573540280858,
     0.7863905325443787,
     0.7797353553374963,
     0.8075237670825907,
     0.8157019294743846
     ])/5

0.7940417876933872